## Technical Analysis Shenanigans

Exploring the TA library and gathering data to use for our MVP

In [24]:
!pip install ta nbimporter wrds cufflinks chart_studio plotly

You should consider upgrading via the 'c:\users\rjsmi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Permission needed to get cufflinks and plotly to work in JupyterLab:
`!jupyter labextension install jupyterlab-plotly`

In [25]:
import nbimporter
from secret.credentials import WRDS_USERNAME

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import cufflinks as cf

import os
import datetime

In [26]:
from ta import add_all_ta_features
from ta.utils import dropna

In [27]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
  
# to get the connection
init_notebook_mode(connected = True)

## Load WRDS data from CSV

In [28]:
df = pd.read_csv('data/sectors/SP500_7yr_daily.csv', sep=',')
print("Number of data points before drop: {}".format(len(df)))
df = dropna(df)
print("Number of data points after drop: {}".format(len(df)))
df["date"] = pd.to_datetime(df["date"])
df.index = df["date"]

Number of data points before drop: 1763
Number of data points after drop: 1762


In [29]:
cf.set_config_file(theme='pearl', world_readable=False)
cf.go_offline()

In [30]:
df["close"].iplot(title='S&P 500 7-yr Close', colors=['red'])

### Plot Sectors

Sector data has been collected using their CUSIP ID (see: data_collection.ipynb). We now aim to plot each sector relative to the S&P 500

In [31]:
import os
from sector import *

In [32]:
stocks = {}
tkr_dict = get_ticker_dict(sectors)

for fname in os.listdir("data/sectors"):
    # Indicates subsector of S&P500
    if fname.startswith("XL") and "_" in fname:
        tkr = fname.split("_")[0]
        stock = Stock(tkr, None, tkr_dict[tkr])
        stock.data = pd.read_csv('data/sectors/'+fname, sep=',')
        
        # Do not wish to dropna, as this cuts out any row with a 0.0 ret or 0.0 retx
        # NaN values will appear during our merge
        # stock.data = dropna(stock.data)
       
        stock.data.index = pd.to_datetime(stock.data["date"], format='%Y-%m-%d') 
        stock.data.pop("date")
        
        # The 'close' and 'retx' column need to be in the first and second columns respectively to match SP5 dataframe (necessary for merging later in train.ipynb)
        close, retx = stock.data['close'], stock.data['retx']
        stock.data.drop(columns=['close', 'retx'], axis=1, inplace=True)
        stock.data.insert(loc=2, column='close', value=close)   # Cols 0 and 1 are ticker and description
        stock.data.insert(loc=3, column='retx', value=retx)
        
        stocks[tkr] = stock

In [33]:
def get_date_range(start_dt, end_dt):
        return pd.date_range(start_dt, end_dt, 
              freq='MS').tolist()

def get_monthly_return(df):
    dates = get_date_range(df.index[0], df.index[-1])
    dates = list((dates[x], dates[x+1]) for x in range(len(dates)-1))
    interval_df = pd.DataFrame()
    ind, ret = [], []
    for dtrng in dates:
        
        # For the special case of % return, we can just look at the price at date1 and the price at date2 and calculate that way
        window = df["close"][(df.index >= dtrng[0]) &  (df.index < dtrng[1])]
        if len(window) == 0:
            print("Empty data within range", dtrng[0], dtrng[1], "for S&P500")
            continue
        
        ret.append(window.iloc[-1] / window.iloc[0] - 1)
            
        # Index will be the starting month (the complete return for that month)
        ind.append(dtrng[0]) 
        
    interval_df["retx"] = ret
    interval_df.index = ind
    
    return interval_df

'''
interval: "monthly", "yearly", or None
'''
def sectplot(col, stock_dict, yTitle="Value ($)", interval=None):
    df = pd.DataFrame()
        #.strftime("%Y-%b")
        # def diff_month(d1, d2):
        #     return 12 * (d2.year - d1.year) + d2.month - d1.month
        
        # def diff_year(d1, d2):
        #     return d2.year - d1.year
        
        # if interval == "monthly":
        #     # Get the 1st date of each month
        #     dates = [(start_dt + datetime.timedelta(days=32*x)).replace(day=1) for x in range(0, diff_month(start_dt, end_dt))]
        # else: # Yearly
        #     dates = [(start_dt + datetime.timedelta(days=365*x)).replace(day=1) for x in range(0, diff_year(start_dt, end_dt))]
        # return dates
    
    for t in stock_dict.keys():
        stock = stocks[t]
        if (interval):    
            # Get the monthly dates
            dates = get_date_range(stock.data.index[0], stock.data.index[-1])
            
            # Convert monthly dates (1, 2, 3, 4) to ((1-2), (2-3), (3-4), ...)
            dates = list((dates[x], dates[x+1]) for x in range(len(dates)-1))
            
            interval_df = pd.DataFrame()
            ind, ret = [], []
            for dtrng in dates:
                
                # For the special case of % return, we can just look at the price at date0 and the price at date2 and calculate that way
                if (col == "retx"):
                    window = stock.data["close"][(stock.data.index >= dtrng[0]) &  (stock.data.index < dtrng[1])]
                    if len(window) == 0:
                        print("Empty data within range", dtrng[0], dtrng[1], "for", stock.desc, t)
                        continue
                    
                    ret.append(window.iloc[-1] / window.iloc[0] - 1)
                else:
                    # Calculate the net column change for that month
                    window = stock.data[col][(stock.data.index >= dtrng[0]) &  (stock.data.index < dtrng[1])]
                    ret.append(window.sum())
                    
                # Index will be the starting month (the complete return for that month)
                ind.append(dtrng[0]) 
                
            interval_df[col] = ret
            interval_df.index = ind
            
            df["{}".format(stock.desc)] = interval_df[col]

        else:
            df["{}".format(stock.desc)] = stock.data[col]

    df.iplot(title='S&P 500 Sectors ({})'.format(col),
                yTitle=yTitle,
                xTitle="Date"
                )
    return df

In [34]:
for k in stocks.keys():
    print(k)
    # stocks[k].data = dropna(stocks[k].data)
    # try/except
    stocks[k].data = add_all_ta_features(
        stocks[k].data,
        open=None,
        high="high",
        low="low",
        close="close",
        volume="vol",
        fillna=True)

XLB


C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



XLC


C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



XLE


...

C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



XLV


C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



XLY


C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\rjsmi\AppData\Local\Programs\Python\Python38\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



In [35]:
sectplot("close", stocks)

,Materials,Communication Services,Energy,Financials,Industrials,Technology,Consumer Staples,Real Estate,Utilities,Health Care,Consumer Discretionary
date,,,,,,,,,,,
2015-01-02,48.650002,NaN,79.529999,24.730000,56.509998,41.270000,48.349998,NaN,47.439999,68.629997,71.629997
2015-01-05,47.410000,NaN,76.239998,24.209999,55.189999,40.639999,48.009998,NaN,46.860001,68.279999,70.260002
2015-01-06,46.990002,NaN,75.120003,23.840000,54.509998,40.150002,47.950001,NaN,46.889999,68.050003,69.559998
2015-01-07,47.520000,NaN,75.279999,24.090000,54.919998,40.490002,48.770000,NaN,47.349998,69.650002,70.660004
2015-01-08,48.650002,NaN,76.970001,24.450001,56.020000,41.380001,49.509998,NaN,47.680000,70.839996,71.720001
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,89.639999,78.870003,56.099998,39.220001,105.019997,176.649994,76.050003,50.939999,70.059998,140.770004,205.169998
2021-12-28,90.160004,78.830002,56.060001,39.240002,105.550003,175.729996,76.480003,51.160000,70.690002,140.399994,205.259995
2021-12-29,90.529999,78.440002,55.700001,39.189999,105.739998,175.880005,76.830002,51.470001,71.070000,141.199997,205.669998


### So what are we predicting?

Well, we want to see the return of each of these sectors relative to the S&P 500. We wish to look at the change in return over an interval, so let's visualize what the % return (or retx) is over time (not cumulative among intervals, only between).

In [36]:
f = sectplot("retx", stocks, interval="monthly", yTitle="Monthly Return (%)")

Now, we need to translate this figure relative to the retx of the S&P500. Thus, what we really wish to predict is:

In [37]:
pred_data = f.copy()
pred_data["S&P500"] = get_monthly_return(df)

# Set all columns relative to S&P500 return and label appropriately
for col in pred_data.columns:
    pred_data[col] -= pred_data["S&P500"]
pred_data.drop("S&P500", axis=1, inplace=True)

In [38]:
pred_data.iplot(title='S&P 500 Sectors Adjusted ({})'.format(col),
                yTitle="Monthly Return vs S&P500 (%)",
                xTitle="Date"
                )

As you can see, the adjusted retx values are slightly more separable than predicting just the return of individual sectors. The basis for each sector's disparity may or may not have some correlation with external factors. The goal is for our model to try to attribute these changes to factors in the movement of the sectors itself.

## Create our groundtruth labels

This monthly return represents, in some capacity, our groundtruth labels. We can further simplify this data to be 1 when outperforming the S&P500, -1 when underperforming the S&P500, and 0 when matching the S&P 500.

In [39]:
f["S&P500"] = get_monthly_return(df)

In [40]:

# Set all columns relative to S&P500 return and label appropriately
for col in f.columns:
    f[col] -= f["S&P500"]
    f[col][f[col] > 0] = 1
    f[col][f[col] <= 0] = 0 #-1
    # f[col][f[col] == 0] = 0
f.drop("S&P500", axis=1, inplace=True)
f["date"] = f.index

# Using the following line of code, we check that there are no 0 entries. Thus, we do not need to worry about there being 
# (f == 0).astype(int).sum(axis=0)

Let's take a look at our data labels!

In [41]:
f

,Materials,Communication Services,Energy,Financials,Industrials,Technology,Consumer Staples,Real Estate,Utilities,Health Care,Consumer Discretionary,date
2015-02-01,1.0,NaN,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,1.0,2015-02-01
2015-03-01,0.0,NaN,1.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,2015-03-01
2015-04-01,1.0,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,2015-04-01
2015-05-01,0.0,NaN,0.0,1.0,0.0,1.0,1.0,NaN,1.0,1.0,0.0,2015-05-01
2015-06-01,0.0,NaN,0.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0,1.0,2015-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-01,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2021-07-01
2021-08-01,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2021-08-01
2021-09-01,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-09-01
2021-10-01,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2021-10-01


## Extract our features and save it as our dataset

#### What kind of TA do we have for our stocks?
Well, let's take a look at our feature list!

In [42]:
stocks["XLK"].data.columns.values[10:]

array(['volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em',
       'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi',
       'volume_nvi', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch',
       'volatility_kcl', 'volatility_kcw', 'volatility_kcp',
       'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
       'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg',
       'trend_vortex_ind_diff', 'trend_trix', 'trend_mass_index',
       'trend_dpo', 'trend_kst', 'trend_kst_sig', 'trend_kst_diff',
       'trend_ichimoku_conv', 'trend_ichimoku_base', 'trend_ichimoku_a',
       'tren

Finally, we save our data with the TA included

In [43]:
if not os.path.exists("data/sectors/TA"):
    os.makedirs("data/sectors/TA")
    
for stock in stocks:
    stocks[stock].data.to_csv("data/sectors/TA/"+stock+"_7yr_daily.csv")

In [44]:
f.to_csv("data/sectors/sector_labels.csv")